In [ ]:
#displaying last 12 months and n months forecasting
import pandas as pd
import numpy as np

from tensorflow.keras.models import load_model
import joblib

!pip install flask-ngrok
from flask import Flask
from flask_ngrok import run_with_ngrok

In [ ]:
table = pd.read_html("https://stat.gov.pl/obszary-tematyczne/ceny-handel/wskazniki-cen/wskazniki-cen-towarow-i-uslug-konsumpcyjnych-pot-inflacja-/miesieczne-wskazniki-cen-towarow-i-uslug-konsumpcyjnych-od-1982-roku/")

In [ ]:
table = table[0].iloc[81:122, 1:14]

In [ ]:
months = [str(x) for x in range(1, 13)]

In [ ]:
table = table.set_axis(['year']+months, axis=1)

In [ ]:
table.iloc[23, 0] = '1999'

In [ ]:
table = table.set_index('year')

In [ ]:
table.info()

<class 'pandas.core.frame.DataFrame'>
Index: 41 entries, 2022 to 1982
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   1       41 non-null     object
 1   2       41 non-null     object
 2   3       41 non-null     object
 3   4       41 non-null     object
 4   5       41 non-null     object
 5   6       41 non-null     object
 6   7       41 non-null     object
 7   8       41 non-null     object
 8   9       40 non-null     object
 9   10      40 non-null     object
 10  11      40 non-null     object
 11  12      40 non-null     object
dtypes: object(12)
memory usage: 4.2+ KB


In [ ]:
df = table.astype(float)

In [ ]:
df = (df - 1000)/10

In [ ]:
df

,1,2,3,4,5,6,7,8,9,10,11,12
year,,,,,,,,,,,,
2022,9.4,8.5,11.0,12.4,13.9,15.5,15.6,16.1,NaN,NaN,NaN,NaN
2021,2.6,2.4,3.2,4.3,4.7,4.4,5.0,5.5,5.9,6.8,7.8,8.6
2020,4.3,4.7,4.6,3.4,2.9,3.3,3.0,2.9,3.2,3.1,3.0,2.4
2019,0.7,1.2,1.7,2.2,2.4,2.6,2.9,2.9,2.6,2.5,2.6,3.4
2018,1.9,1.4,1.3,1.6,1.7,2.0,2.0,2.0,1.9,1.8,1.3,1.1
2017,1.7,2.2,2.0,2.0,1.9,1.5,1.7,1.8,2.2,2.1,2.5,2.1
2016,-0.9,-0.8,-0.9,-1.1,-0.9,-0.8,-0.9,-0.8,-0.5,-0.2,0.0,0.8
2015,-1.4,-1.6,-1.5,-1.1,-0.9,-0.8,-0.7,-0.6,-0.8,-0.7,-0.6,-0.5
2014,0.5,0.7,0.7,0.3,0.2,0.3,-0.2,-0.3,-0.3,-0.6,-0.6,-1.0


In [ ]:
df = df.iloc[::-1]

In [ ]:
df = pd.DataFrame(df.stack()).iloc[-24:]

In [ ]:
new_dates = pd.date_range(start='1982-01', end='2022-07', freq='MS')
df.index = new_dates

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive/Colab Notebooks/Flask/

In [ ]:
model = load_model("simpleRNN.h5")
scaler = joblib.load("RNNscaler.pkl")

In [ ]:
forecast = []

periods = 12

data = df.copy()

for i in range(periods):
  eval_batch = data[-24:]
  current_batch = eval_batch.reshape((1, 24, 1))   
  current_pred = model.predict(current_batch)[0]
  forecast.append(current_pred) 
  data = np.append(data, current_pred[0])
forecast = scaler.inverse_transform(forecast)
forecast_index = pd.date_range(start='2022-08', periods=periods, freq='MS')
forecast_df = pd.DataFrame(data=forecast, index=forecast_index, columns=['Forecast'])

#Flask app

In [ ]:
# app = Flask(__name__)
# run_with_ngrok(app)   
  
# @app.route("/")
# def home():
#     return "<h1>TEST</h1>"
    
# app.run()